In [ ]:
#!pip install python-dotenv
#!pip install whisper
#!pip install ffmpeg-python
#!pip install google-generativeai
#!pip install PyHackMD
#!pip install tqdm

In [1]:
from dotenv import load_dotenv
import whisper
import ffmpeg
import os
from IPython.display import display, Markdown
import google.generativeai as genai

import pandas as pd
from tqdm.auto import tqdm
from pathlib import Path

load_dotenv()  # This loads the variables from .env into the environment


def video_to_organized_md(api_key, s_path, model_name="base", language=None, temp=0.5, max_tokens=4096, prompt="", output=None):
    
    try:
        # Configure Google API
        genai.configure(api_key=api_key)

        # Determine the base name for output files early to ensure it's available for all paths
        base_name = os.path.splitext(os.path.basename(s_path))[0]
        organized_output = os.path.join(os.path.dirname(s_path), f"{base_name}_organized.txt") if not output else output

        file_ext = os.path.splitext(s_path)[1].lower()
        transcription = ""

        # Check if input is a text file
        if file_ext == '.txt':
            print(f"Reading from text file {s_path}...")
            with open(s_path, 'r') as file:
                transcription = file.read()
        else:
            # Process for non-text files
            audio_path = s_path
            if file_ext not in ['.wav', '.mp3', '.flac', '.m4a']:
                # Convert video to audio if not an accepted format
                audio_path = os.path.splitext(s_path)[0] + ".wav"
                print("Converting video to audio...")
                (
                    ffmpeg
                    .input(s_path)
                    .output(audio_path, ac=1, ar="16k")
                    .run(quiet=True, overwrite_output=True)
                )
                print(f"Audio extracted and saved to {audio_path}")

            # Load the Whisper model and transcribe
            print(f"Loading model '{model_name}' for transcription...")
            whisper_model = whisper.load_model(model_name)
            print("Transcribing...")
            result = whisper_model.transcribe(audio_path, language=language, without_timestamps=True)
            transcription = result["text"]

            # Save the transcription to a text file
            t_path = os.path.join(os.path.dirname(s_path), f"{base_name}.txt")
            with open(t_path, 'w') as text_file:
                text_file.write(transcription)
            print(f"Transcription completed and saved to {t_path}")

        # Organize text using Gemini Pro
        full_prompt = prompt + "\n\n" + transcription
        model = genai.GenerativeModel(model_name="gemini-pro", 
                                      generation_config={"temperature": temp, "top_p": 1, "top_k": 1, "max_output_tokens": max_tokens}, 
                                      safety_settings=[{"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_ONLY_HIGH"}, 
                                                       {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_ONLY_HIGH"}, 
                                                       {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_ONLY_HIGH"}, 
                                                       {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_ONLY_HIGH"}])
        print("Organizing text...")
        response = model.generate_content([full_prompt])

        # Save and display the organized text
        with open(organized_output, 'w') as file:
            file.write(response.text)
        print(f"Organized text saved to {organized_output}")
        print()
        display(Markdown(response.text))

    except Exception as e:
        print(f"An error occurred: {e}")


# Assume video_to_organized_md is defined as previously, but adjusted for these parameters

def batch_video_to_md(directory, api_key, model_name, language, temp, max_tokens, prompt):
    
    if not os.path.isdir(directory):
        print(f"The provided path {directory} is not a directory.")
        return

    # Collect all files with preference for processing
    file_extensions = ['.txt', '.wav','.m4a', '.mp4'] # You can modify this in your favor
    all_files = list(Path(directory).rglob('*'))
    # Filtering and prioritizing files
    files_to_process = {}
    for file in all_files:
        if file.suffix.lower() in file_extensions:
            base = str(file.parent / file.stem)
            if base not in files_to_process or file_extensions.index(file.suffix.lower()) < file_extensions.index(files_to_process[base].suffix.lower()):
                files_to_process[base] = file

    # Creating a DataFrame to display processing files
    data = [[file.stem, file, file.suffix.lower()] for file in files_to_process.values()]
    df = pd.DataFrame(data, columns=['Base Name', 'File Path', 'Type'])
    display(df)

    # Processing files with progress bar
    for _, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing Files"):
        try:
            # Correct approach to append "_organized.md" to the base name while keeping the directory structure intact
            file_path = str(row['File Path'])
            base_path_without_ext = file_path.rsplit('.', 1)[0]  # This splits the path and removes the extension
            output = base_path_without_ext + "_organized.md"  # Now you append your custom suffix correctly
            
            # Calling the function with all necessary parameters
            video_to_organized_md(api_key, file_path, model_name, language, temp, max_tokens, prompt, output)
        except Exception as e:
            print(f"Error processing {file_path}: {e}")


###################################################################################

from PyHackMD import API

def hackmd(hackmd_api_key, path):
    # Initialize the API object with the provided API key
    api = API(hackmd_api_key)

    # Walk through all directories and subdirectories in the given path to find markdown files
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".md"):
                file_path = os.path.join(root, file)
                try:
                    with open(file_path, 'r', encoding='utf-8') as md_file:
                        md_content = md_file.read()

                        # Attempt to create a new note for each Markdown file
                        try:
                            data = api.create_note(title=os.path.splitext(file)[0])
                            note_id = data.get('id')
                            #print(note_id)
                            api.update_note(note_id, content=md_content)
                            # Adjust permissions as needed. The parameters below are placeholders.
                            # Replace or remove the permission parameters based on actual API usage or omit them if unnecessary.
                            print(f"Successfully uploaded {file}")
                        except Exception as api_error:
                            print(f"Failed to upload {file} due to API error: {api_error}")
                except IOError as io_error:
                    print(f"Could not read file {file_path}: {io_error}")

In [ ]:
# Execute this function to convert video/audio/text into markdown
api_key = os.getenv('api_key')

model_name = "base" 
language = "en"
temp = 0.5
max_tokens = 8192

prompt = "Please use the following audio-to-text transcription to organize the information of the speech in h2 markdown. Since it is an AI transcription, errors could happen, please refer to the contexts of the entire speech. Make sure ALL the details are logically categorized, and preserved, especially the medical related details. \n\n Please use h2 markdown and must include the topic of the speech, speaker's name and info, the content of the speech, and the summary of the speech."

directory =  os.getenv('directory') # Replace with the actual directory path
batch_video_to_md(directory, api_key, model_name, language, temp, max_tokens, prompt)

In [ ]:
# Execute this function to batch upload the converted markdown notes to HackMD
hackmd_api_key = os.getenv('hackmd_api_key')
directory =  os.getenv('directory')
hackmd(hackmd_api_key, directory)